### Imports

In [4]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [5]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

In [16]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.model_helper import *
from model_scripts.pretrained_temporal_feature_extraction import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.clustering import *
from model_scripts.subpatch_extraction import *
from evaluation_scripts.result_visualiser import *
from evaluation_scripts.evaluation_helper import *
from Pipeline.pre_processing_pipeline import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import preprocessing_config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

## Dataset prep: B10

### Loading the pre-processed data

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)

Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

In [ ]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, images_visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'allbands')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, images_visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'allbands')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

### Create Sub-Patches

In [ ]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
eval_subpatches, eval_subpatch_coords = non_overlapping_sliding_window(patch_tensor_eval, field_numbers_eval, patch_size=config.subpatch_size)
train_subpatches.shape, eval_subpatches.shape

(torch.Size([39042, 7, 10, 5, 5]), torch.Size([867, 7, 10, 5, 5]))

Get field numbers and co-ordinates as string

In [ ]:
train_coord_dataloader = get_string_fielddata(train_subpatch_coords)
eval_coord_dataloader = get_string_fielddata(eval_subpatch_coords)
train_coord_dataloader[0]

'1167134.0_25_25'

### Split Unlabeled data into 'train' and 'test' and create  Data Loaders

In [ ]:
train_subpatches_dl, test_subpatches, train_field_numbers, test_field_numbers = train_test_split(
    train_subpatches, train_coord_dataloader, test_size=1-config.ae_train_test_ratio, random_state=42
)

dataloader_train = create_data_loader(train_subpatches_dl, train_field_numbers, batch_size=config.ae_batch_size, shuffle=True)
dataloader_test = create_data_loader(test_subpatches, test_field_numbers, batch_size=config.ae_batch_size, shuffle=False)
dataloader_eval = create_data_loader(eval_subpatches, eval_coord_dataloader, batch_size=config.ae_batch_size, shuffle=False)

/home/k64835/Master-Thesis-SITS/Modeling/model_scripts/dataset_creation.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).permute(0, 2, 1, 3, 4)   # (N, T, C, H, W) -> (N, C, T, H, W)


## Pre-trained Model 1: Resnet 3D trained on Video Data

### Extract Features

In [ ]:
device = 'cuda'

In [32]:
resnet3d_extractor = ResNet3DFeatureExtractor()
resnet3d_features_train, train_coord_dl = extract_features(resnet3d_extractor, dataloader_train, device)
print("ResNet3D Extracted Features Shape:", resnet3d_features_train.shape)  

/home/k64835/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/k64835/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet3D Extracted Features Shape: torch.Size([31233, 512])


In [33]:
resnet3d_features_test, test_coord_dl = extract_features(resnet3d_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", resnet3d_features_test.shape)  

ResNet3D Extracted Features Shape: torch.Size([7809, 512])


In [34]:
resnet3d_features_eval, eval_coord_dl = extract_features(resnet3d_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", resnet3d_features_eval.shape)  

ResNet3D Extracted Features Shape: torch.Size([867, 512])


### K-means + Evaluation

In [35]:
kmeans = train_kmeans_patches(resnet3d_features_train.cpu(), n_clusters=2, random_state=12)

train_patch_predictions = kmeans.predict(resnet3d_features_train.reshape(resnet3d_features_train.size(0), -1).numpy().astype(np.float32))
test_patch_predictions = kmeans.predict(resnet3d_features_test.reshape(resnet3d_features_test.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(resnet3d_features_eval.reshape(resnet3d_features_eval.size(0), -1).numpy().astype(np.float32))

/home/k64835/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [36]:
# Assign field labels
threshold = 0.5
train_field_labels = assign_field_labels_ae(train_coord_dl, train_patch_predictions, threshold)
test_field_labels = assign_field_labels_ae(test_coord_dl, test_patch_predictions, threshold)
eval_field_labels = assign_field_labels_ae(eval_coord_dl, eval_patch_predictions, threshold)

In [37]:
accuracy, report, x_y_coords = evaluate_test_labels_ae(eval_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

Test Accuracy: 0.5245901639344263
              precision    recall  f1-score   support

           0       0.40      0.23      0.29        26
           1       0.57      0.74      0.64        35

    accuracy                           0.52        61
   macro avg       0.48      0.49      0.47        61
weighted avg       0.49      0.52      0.49        61



## Pre-trained Models: ConvLSTM -- remove

In [39]:
convlstm_extractor = ConvLSTMFeatureExtractor()
convlstm_features_train, train_coord_dl = extract_features(convlstm_extractor, dataloader_train, device)
print("ConvLSTM Extracted Features Shape:", convlstm_features_train.shape)


In [ ]:
convlstm_features_test, test_coord_dl = extract_features(convlstm_extractor, dataloader_test, device)
print("ResNet3D Extracted Features Shape:", convlstm_features_test.shape)  

ResNet3D Extracted Features Shape: torch.Size([7809, 512])


In [ ]:
convlstm_features_eval, eval_coord_dl = extract_features(convlstm_extractor, dataloader_eval, device)
print("ResNet3D Extracted Features Shape:", convlstm_features_eval.shape)  

ResNet3D Extracted Features Shape: torch.Size([867, 512])
